In [1]:
#Running the core utilities notebook with all validation and loading functions
%run transform_utils.ipynb

In [6]:
#load data
transaction_silver_df, transaction_silver_df_counts = read_transactions_csv(transactions_bronze_path)
transaction_silver_df.head()

,location_id,date,transaction_id,profit
0,008,2019-01-02,001,26.89
1,008,2019-01-02,002,24.74
2,008,2019-01-02,003,31.36
3,008,2019-01-02,004,27.06
4,008,2019-01-02,005,29.51


In [11]:
# validate if the sum of the rows in individual files is equal to the number of rows in the final df 
assert len(transaction_silver_df) == sum(transaction_silver_df_counts)
sum(transaction_silver_df_counts)

85347

In [12]:
# since there are no transformations I am directly writing - leaving room for future transformations
file_name = 'transaction_silver_data.csv'
write_to_path_silver(transaction_silver_df,silver_transaction_path,file_name)

DataFrame saved to ../Submission/silver_data/silver_transactions/transaction_silver_data.csv


In [27]:
# Find duplicate rows based on 'transaction_id', 'location_id', and 'date'
duplicates = transaction_silver_df.duplicated(subset=['transaction_id', 'location_id', 'date'], keep=False)

# Filter the DataFrame to show only the duplicate rows
duplicate_rows = transaction_silver_df[duplicates]

# Display the duplicate rows
print(duplicate_rows)


Empty DataFrame
Columns: [location_id, date, transaction_id, profit]
Index: []


In [34]:
#open db connection
conn = sqlite3.connect('tubing.db')
cursor = conn.cursor()

In [35]:
#load rows to table
transaction_silver_df.to_sql('transactions', conn, if_exists='append', index=False)

85347

In [41]:
# Drop the temporary table


query = """
SELECT * FROM transactions LIMIT 10
"""
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Print the results
if results:
    print("Fetched rows:")
    for row in results:
        print(row)
else:
    print("No rows fetched. The table might be empty.")
# Close the connection



Fetched rows:
('001', '008', '2019-01-02T00:00:00', 26.89)
('002', '008', '2019-01-02T00:00:00', 24.74)
('003', '008', '2019-01-02T00:00:00', 31.36)
('004', '008', '2019-01-02T00:00:00', 27.06)
('005', '008', '2019-01-02T00:00:00', 29.51)
('006', '008', '2019-01-02T00:00:00', 27.52)
('007', '008', '2019-01-02T00:00:00', 26.59)
('008', '008', '2019-01-02T00:00:00', 28.4)
('009', '008', '2019-01-02T00:00:00', 28.42)
('010', '008', '2019-01-02T00:00:00', 25.11)


In [32]:
conn.close()